<p align=center><img src=https://d31uz8lwfmyn8g.cloudfront.net/Assets/logo-henry-white-lg.png><p>

# <h1 align=center> **PROYECTO INDIVIDUAL Nº1** </h1>
# <h1 align=center>**`@autor: Lesmen Gerardo García Pernia`**</h1>
# <h1 align=center>**`Machine Learning Operations (MLOps)`**</h1>





Definición de librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import re
import ast

Carga de los Datos

In [2]:
df_movies = pd.read_csv('../DataSetsPI/movies_dataset.csv')
df_credits = pd.read_csv('../DataSetsPI/credits.csv')

C:\Users\lesme\AppData\Local\Temp\ipykernel_3860\1434914061.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('../DataSetsPI/movies_dataset.csv')


In [3]:
df_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [4]:
### Validando las columnas
df_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
### Revisando los tipos de datos de las columnas y cantidad total
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [7]:
### Validando columnas numericas
df_movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [8]:
### Visualizando columnas Anidadas. Se observan 5 columnas anidadas.
df_movies.iloc[4]

adult                                                                False
belongs_to_collection    {'id': 96871, 'name': 'Father of the Bride Col...
budget                                                                   0
genres                                      [{'id': 35, 'name': 'Comedy'}]
homepage                                                               NaN
id                                                                   11862
imdb_id                                                          tt0113041
original_language                                                       en
original_title                                 Father of the Bride Part II
overview                 Just when George Banks has recovered from his ...
popularity                                                        8.387519
poster_path                               /e64sOI48hQXyru7naBFyssKFxVd.jpg
production_companies     [{'name': 'Sandollar Productions', 'id': 5842}...
production_countries     

## Requerimiento # Nro. 1

+ Algunos campos, como **`belongs_to_collection`**, **`production_companies`** y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder  y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

### Desanidando los siguientes columnas:

<ul>
<li> 1) belongs_to_collection (Diccionario).</li>
<li> 2) genres (Lista de Diccionario). </li>
<li> 3) production_companies (Lista de Diccionario).</li>
<li> 4) production_countries (Lista de Diccionario).</li>
<li> 5) spoken_languages (Lista de Diccionario).</li>
</ul>

### Creación de Funciones para Desanidar los campos anteriores:

Se procede a crear dos Funciones para Desanidar estas 5 columnas anidadas. La primera función ***(Extraer_Valor_Dicc)***, para extraer de la columna 'belongs_to_collection' sus campos en forma de diccionarios. Y la segunda función ***(Extraer_Valor_List)***, para extraer del resto de las columnas que contienen sus campos como una lista de diccionarios.

1 ) Funcion: Extraer_Valor_Dicc:<Br>
*Funcion para evaluar columnas tipo Diccionario y extraer el valor asociado a la clave 'name'; devolviendo dicho valor.*

In [9]:
# Función para extraer el valor de la clave 'name', de la columna diccionario: 
def Extraer_Valor_Dicc(value):
    try:
        diccionario = ast.literal_eval(value)  #Evaluamos cada atributo como un diccionario de Python
        if isinstance(diccionario, dict): #Si el valor es un diccionario, extrae el valor asociado a la clave 'name', para luego retornarlo
            return diccionario.get('name', '')
    except (ValueError, SyntaxError): #Si no es posible evaluarlo como un diccionario, capturamos las excepciones (ValueError o SyntaxError) y devolvemos una cadena vacía.
        pass
    return ''  # En caso de no ser un diccionario, se devuelve una cadena vacía.

2  ) Funcion: Extraer_Valor_Lista:<Br>
*Funcion para evaluar columnas tipo Listas de Diccionario y extraer el valor asociado a la clave 'name' de cada diccionario, concatenándolos en una sola cadena.* 

In [11]:
# Función para extraer el valor de la clave 'name', de la columna lista de diccionarios: genres
def Extraer_Valor_List(value):
    try:
        lista_de_diccionarios = ast.literal_eval(value) ##Evaluamos cada atributo como una lista de diccionario de Python
        if isinstance(lista_de_diccionarios, list): #Si el valor es una lista de diccionario, extrae el valor asociado a la 
                                                    #clave 'name' de cada diccionario y lo concatena en una sola cadena.
            names = [dic.get('name', '') for dic in lista_de_diccionarios if isinstance(dic, dict)]
            return ', '.join(names)
    except (ValueError, SyntaxError):
        pass
    return ''  # Si no es posible evaluarlo como una lista de diccionarios, se devuelve una cadena vacía.

### Desanidando las 5 columnas, todas por la clave-valor. Devolviendo el valor de cada clave.

**`1) belongs_to_collection`**

In [12]:
# Aplicar la función para extraer el valor 'name' y crear la nueva columna 'Collection'
df_movies['collection'] = df_movies['belongs_to_collection'].apply(Extraer_Valor_Dicc)

**`2) genes`**

In [13]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'genres'
df_movies['genre'] = df_movies['genres'].apply(Extraer_Valor_List)

**`3) production_companies`**

In [14]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'Collection'
df_movies['companies'] = df_movies['production_companies'].apply(Extraer_Valor_List)

**`4) production_countries`**

In [15]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'Collection'
df_movies['countries'] = df_movies['production_countries'].apply(Extraer_Valor_List)

**`5) spoken_languages`**

In [16]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'Collection'
df_movies['languages'] = df_movies['spoken_languages'].apply(Extraer_Valor_List)

In [17]:
### Validando las nuevas 5 columnas creadas
df_movies[['collection', 'genre', 'companies', 'countries', 'languages' ]]

,collection,genre,companies,countries,languages
0,Toy Story Collection,"Animation, Comedy, Family",Pixar Animation Studios,United States of America,English
1,,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français"
2,Grumpy Old Men Collection,"Romance, Comedy","Warner Bros., Lancaster Gate",United States of America,English
3,,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,United States of America,English
4,Father of the Bride Collection,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America,English
...,...,...,...,...,...
45461,,"Drama, Family",,Iran,فارسی
45462,,Drama,Sine Olivia,Philippines,
45463,,"Action, Drama, Thriller",American World Pictures,United States of America,English
45464,,,Yermoliev,Russia,


In [18]:
### Listando todas las columnas
df_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'collection', 'genre', 'companies',
       'countries', 'languages'],
      dtype='object')

In [20]:
### Total de Registros y Columnas
df_movies.shape

(45466, 29)

In [21]:
### Realizamos backup de todos los cambios realizados del Requerimiento Nro. 1
df_movies_bck01 = df_movies.copy()

## Requerimiento # Nro. 2

+ Los valores nulos de los campos **`revenue`**, **`budget`** deben ser rellenados por el número **`0`**.

In [22]:
print("Cantidad de Valores Nulos de 'revenue':",df_movies['revenue'].isna().sum())
print("Cantidad de Valores Nulos de 'budget':",df_movies['budget'].isna().sum())

Cantidad de Valores Nulos de 'revenue': 6
Cantidad de Valores Nulos de 'budget': 0


In [23]:
df_movies.info()

#Efectivamente validamos que faltan 6 datos a la columna 'revenue'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [24]:
### Validando que es una columna númerica
df_movies['revenue']

0        373554033.0
1        262797249.0
2                0.0
3         81452156.0
4         76578911.0
            ...     
45461            0.0
45462            0.0
45463            0.0
45464            0.0
45465            0.0
Name: revenue, Length: 45466, dtype: float64

In [25]:
# Se procede a sustituir los 6 valores nulos en la columna "revenue", con cero (0).
df_movies['revenue'] = df_movies['revenue'].fillna(0)

# Otra forma de hacerlo: 
# df_movies2['revenue'].fillna(0,inplace=True)

# Validamos que ya no existan valores nulos:
df_movies['revenue'].isna().sum()

0

In [26]:
### Continuamos con la columna 'budget'
### Validamos que tipo de dato es.
df_movies['budget'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 45466 entries, 0 to 45465
Series name: budget
Non-Null Count  Dtype 
--------------  ----- 
45466 non-null  object
dtypes: object(1)
memory usage: 355.3+ KB


In [27]:
### Esta definida como Objeto. Validamos el dato directamente en la columna y nos damos cuenta que es númerico: 
df_movies['budget']

0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45461           0
45462           0
45463           0
45464           0
45465           0
Name: budget, Length: 45466, dtype: object

In [28]:
### Validando si hay NaN:
df_movies['budget'].isna().sum()

0

In [29]:
### La columna 'budget' esta definida como Objeto, pero es Númerico.
### Convertiendo la columna de tipo Objeto a tipo Númerico, y los que no sean numericos los convierte en NaN:

df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')

### Validano los cambios realizados a númerico y luego si se h acreado algun NaN:
df_movies['budget'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 45466 entries, 0 to 45465
Series name: budget
Non-Null Count  Dtype  
--------------  -----  
45463 non-null  float64
dtypes: float64(1)
memory usage: 355.3 KB


In [30]:
# Validando si aparecieron NaN: (por el cambio realizado en el paso anterior)
df_movies['budget'].isna().sum()  

3

In [31]:
# Llevamos los 3 valores NaN de la columna "budget" a cero (0).
df_movies['budget'] = df_movies['budget'].fillna(0)

### Validando que ya no hay NaN
df_movies['budget'].isna().sum()

0

In [32]:
### Validamos nuevamente nuestro DataSet:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  float64
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [33]:
df_movies.describe()

,budget,revenue,runtime,vote_average,vote_count
count,4.546600e+04,4.546600e+04,45203.000000,45460.000000,45460.000000
mean,4.224300e+06,1.120787e+07,94.128199,5.618207,109.897338
std,1.742359e+07,6.432813e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,0.000000e+00,107.000000,6.800000,34.000000
max,3.800000e+08,2.787965e+09,1256.000000,10.000000,14075.000000


In [34]:
### Realizamos backup de todos los cambios realizados del Requerimiento Nro. 2
df_movies_bck02 = df_movies.copy()

## Requerimiento Nro. 3

+ Los valores nulos del campo **`release_date`** deben eliminarse.

In [35]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  float64
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [36]:
### La columna 'release_date' esta definida de tipo Object, segun el comando anterior.

### Validando si es o no, si es de tipo Objeto:
df_movies['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [37]:
### Validamos si tiene NaN:
df_movies['release_date'].isna().sum()

87

In [38]:
### Efectivamente hay 87 NaN.
45379 + 87

45466

In [39]:
### Primero convertimos a datetime la columna 'release_date'
### Y forzamos a convertir en NaN a todos los valores que den error (errors='coerce).
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')

### Validamos nuevamente la cantidad de valores NaN
df_movies['release_date'].isna().sum()

90

In [242]:
### Efectivamente, ahora el total de NaN en la columna 'release_date' es 90.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  45466 non-null  object        
 1   belongs_to_collection  4494 non-null   object        
 2   budget                 45466 non-null  float64       
 3   genres                 45466 non-null  object        
 4   homepage               7782 non-null   object        
 5   id                     45466 non-null  object        
 6   imdb_id                45449 non-null  object        
 7   original_language      45455 non-null  object        
 8   original_title         45466 non-null  object        
 9   overview               44512 non-null  object        
 10  popularity             45461 non-null  object        
 11  poster_path            45080 non-null  object        
 12  production_companies   45463 non-null  object        
 13  p

In [40]:
### Ahora si procedemos a eliminar los valores NaN = 87 +3 = 90, de la columna 'release_date'.
df_movies.dropna(subset=['release_date'], inplace=True)

#Verificamos que no existan NaT
df_movies['release_date'].isnull().sum()

0

In [41]:
### Validamos nuevamente:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  45376 non-null  object        
 1   belongs_to_collection  4488 non-null   object        
 2   budget                 45376 non-null  float64       
 3   genres                 45376 non-null  object        
 4   homepage               7766 non-null   object        
 5   id                     45376 non-null  object        
 6   imdb_id                45362 non-null  object        
 7   original_language      45365 non-null  object        
 8   original_title         45376 non-null  object        
 9   overview               44435 non-null  object        
 10  popularity             45376 non-null  object        
 11  poster_path            45037 non-null  object        
 12  production_companies   45376 non-null  object        
 13  produc

In [42]:
### Realizamos backup de todos los cambios realizados del Requerimiento Nro. 3
df_movies_bck03 = df_movies.copy()

## Requerimiento Nro. 4

+ De haber fechas, deberán tener el formato **`AAAA-mm-dd`**, además deberán crear la columna **`release_year`** donde extraerán el año de la fecha de estreno.

In [43]:
### Validan el tipo de columna 'release_date' y sus datos
print(df_movies['release_date'])
df_movies['release_date'].info()


0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45460   1991-05-13
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45376, dtype: datetime64[ns]
<class 'pandas.core.series.Series'>
Index: 45376 entries, 0 to 45465
Series name: release_date
Non-Null Count  Dtype         
--------------  -----         
45376 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 709.0 KB


In [44]:
#Verificamos que todos los registros tengan el mismo formato AAAA-MM-DD
df_movies['release_date'] = pd.to_datetime(df_movies['release_date']).dt.strftime('%Y-%m-%d')

In [45]:
### Como efectivamente todos los registros de la columna 'release_date' tienen el mismo formato, pasamos a crear la columna de años:

#Creamos la columna 'release_year' que va a contener el año de la columna 'release_date'
df_movies['release_year'] = pd.to_datetime(df_movies['release_date']).dt.year

### Validamos la nueva columna de 'release_year'
print(df_movies[['release_date','release_year']].head(5))
df_movies[['release_date','release_year']].tail(5)

  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
3   1995-12-22          1995
4   1995-02-10          1995


,release_date,release_year
45460,1991-05-13,1991
45462,2011-11-17,2011
45463,2003-08-01,2003
45464,1917-10-21,1917
45465,2017-06-09,2017


In [46]:
### Realizamos backup de todos los cambios realizados del Requerimiento Nro. 4
df_movies_bck04 = df_movies.copy()

## Requerimiento Nro. 5

+ Crear la columna con el retorno de inversión, llamada **`return`** con los campos **`revenue`** y **`budget`**, dividiendo estas dos últimas **`revenue / budget`**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor **`0`**.

In [47]:
### Validando los tipos de las columnas
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45376 non-null  object 
 1   belongs_to_collection  4488 non-null   object 
 2   budget                 45376 non-null  float64
 3   genres                 45376 non-null  object 
 4   homepage               7766 non-null   object 
 5   id                     45376 non-null  object 
 6   imdb_id                45362 non-null  object 
 7   original_language      45365 non-null  object 
 8   original_title         45376 non-null  object 
 9   overview               44435 non-null  object 
 10  popularity             45376 non-null  object 
 11  poster_path            45037 non-null  object 
 12  production_companies   45376 non-null  object 
 13  production_countries   45376 non-null  object 
 14  release_date           45376 non-null  object 
 15  revenue

In [48]:
### Validando que no hay NaN, en ninguna de las dos columnas:
print(df_movies['budget'].isna().sum())
df_movies['revenue'].isna().sum()

0


0

In [49]:
### Validando datos
df_movies[['budget','revenue']].describe()

,budget,revenue
count,4.537600e+04,4.537600e+04
mean,4.232604e+06,1.123010e+07
std,1.743986e+07,6.438996e+07
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,3.800000e+08,2.787965e+09


In [50]:
### Se realiza esta conversion para detectar la existencia de algun dato, con un posible error. (Por si acaso !!!!)
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce')

In [51]:
### Creando la columna con la formula del retorno de inversión 'return'
df_movies['return'] = df_movies['revenue']/df_movies['budget']

### Eliminando valores NaN
df_movies['return'].fillna(0, inplace=True)

### Visualizando las columnas
print(df_movies[['revenue', 'budget','return']].head(10))
df_movies[['revenue', 'budget','return']].tail(10)


       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0        inf
5  187436818.0  60000000.0   3.123947
6          0.0  58000000.0   0.000000
7          0.0         0.0   0.000000
8   64350171.0  35000000.0   1.838576
9  352194034.0  58000000.0   6.072311


,revenue,budget,return
45455,0.0,0.0,0.0
45456,0.0,0.0,0.0
45457,0.0,0.0,0.0
45458,0.0,0.0,0.0
45459,0.0,0.0,0.0
45460,0.0,0.0,0.0
45462,0.0,0.0,0.0
45463,0.0,0.0,0.0
45464,0.0,0.0,0.0
45465,0.0,0.0,0.0


In [52]:
### Se observan divisiones por CERO, que son colocadas como: inf ( Infinito).

### Se procede a convertir los valores INFINITOS a CERO:
df_movies['return'] = df_movies['return'].replace([np.inf, -np.inf], 0)

### Visualizando las columnas nuevamente:
print(df_movies[['revenue', 'budget','return']].head(10))
df_movies[['revenue', 'budget','return']].tail(10)

       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000
5  187436818.0  60000000.0   3.123947
6          0.0  58000000.0   0.000000
7          0.0         0.0   0.000000
8   64350171.0  35000000.0   1.838576
9  352194034.0  58000000.0   6.072311


,revenue,budget,return
45455,0.0,0.0,0.0
45456,0.0,0.0,0.0
45457,0.0,0.0,0.0
45458,0.0,0.0,0.0
45459,0.0,0.0,0.0
45460,0.0,0.0,0.0
45462,0.0,0.0,0.0
45463,0.0,0.0,0.0
45464,0.0,0.0,0.0
45465,0.0,0.0,0.0


In [53]:
### Error de division por CERO corregido.

### Validamos nuevamente el Dataset:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45376 non-null  object 
 1   belongs_to_collection  4488 non-null   object 
 2   budget                 45376 non-null  float64
 3   genres                 45376 non-null  object 
 4   homepage               7766 non-null   object 
 5   id                     45376 non-null  object 
 6   imdb_id                45362 non-null  object 
 7   original_language      45365 non-null  object 
 8   original_title         45376 non-null  object 
 9   overview               44435 non-null  object 
 10  popularity             45376 non-null  object 
 11  poster_path            45037 non-null  object 
 12  production_companies   45376 non-null  object 
 13  production_countries   45376 non-null  object 
 14  release_date           45376 non-null  object 
 15  revenue

In [54]:
### Realizamos backup de todos los cambios realizados del Requerimiento Nro. 5
df_movies_bck05 = df_movies.copy()

## Requerimiento Nro. 6

+ Eliminar las columnas que no serán utilizadas, **`video`**,**`imdb_id`**,**`adult`**,**`original_title`**,**`poster_path`** y **`homepage`**.

In [55]:
### Total de Columnas
print("Total de Columnas=",df_movies.columns.value_counts().sum())

### Cantidad de columnas a eliminar por el Requeriiento Nro.6
print("Nro de Columnas a eliminar el el Requerimienti Nro.6 = 7 ")

### Se procede a eliminar las 7 columnas:
df_movies.drop(columns=['video', 'imdb_id','adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], inplace=True)

### Columnas Restantes:
print("Columnas Restantes =",df_movies.columns.value_counts().sum())

Total de Columnas= 31
Nro de Columnas a eliminar el el Requerimienti Nro.6 = 7 
Columnas Restantes = 24


In [56]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  float64
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  object 
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  title  

In [57]:
### Validando Columnas:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'collection', 'genre', 'companies', 'countries', 'languages',
       'release_year', 'return'],
      dtype='object')

In [74]:
### Validando Columnas y tipos de datos:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  float64
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  object 
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

In [75]:
### Total de Registros y Columnas:
df_movies.shape

(45376, 24)

In [76]:
### Realizamos backup antes de eliminar las columnas anidadas:
df_movies_bck07 = df_movies.copy()

In [86]:
### Se procede a eliminar las 5 Columnas que se estan anidadas:
df_movies.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages' ], inplace=True)

### Reseteando los Indices del DataSet, por todos los cambios realizados:
df_movies.reset_index(drop=True, inplace=True)

### Validan nuevamente el total de registros y columnas:
df_movies.shape

(45376, 19)

In [87]:
df_movies

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,collection,genre,companies,countries,languages,release_year,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,Toy Story Collection,"Animation, Comedy, Family",Pixar Animation Studios,United States of America,English,1995,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français",1995,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,Grumpy Old Men Collection,"Romance, Comedy","Warner Bros., Lancaster Gate",United States of America,English,1995,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,United States of America,English,1995,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,Father of the Bride Collection,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America,English,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45371,0.0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,NaN,Robin Hood,5.7,,"Drama, Action, Romance","Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...",English,1991,0.000000
45372,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,Century of Birthing,9.0,,Drama,Sine Olivia,Philippines,,2011,0.000000
45373,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,,"Action, Drama, Thriller",American World Pictures,United States of America,English,2003,0.000000
45374,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,,,Yermoliev,Russia,,1917,0.000000


In [88]:
### Realizamos backup de todos los cambios realizados  hasta estos momentos:
df_movies_bck08 = df_movies.copy()

In [89]:
### Realizando un Backup en CSV, del DataSet con los cambios relizados por requerimientos iniciales (df_movies_RI)
df_movies.to_csv('../DataSetPI-Cleaning/movies_dataset_RI.csv', index=False, header=True, sep=';')

In [90]:
### Realizando un Backup en Excel, del DataSet con los cambios relizados por requerimientos iniciales (df_movies_RI)
df_movies.to_excel('../DataSetPI-Cleaning/df_movies_dataset-RI.xlsx', index=False, header=True)

### Fin de los Requerimiento Iniciales

## Fin de la Primera Parte

# Inicio de las APIs